I learned from [**How to Create a Photo Mosaic in Python**](https://towardsdatascience.com/how-to-create-a-photo-mosaic-in-python-45c94f6e8308)

In [13]:
#import libs

import glob
from PIL import Image
from scipy import spatial
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# define the path of tiles and main image

main_photo_path = 'photos/main/main.jpg'
tile_photos_path = 'photos/tiles/*'
tile_size = (50,50)

In [3]:
# get all tiles
tile_paths = []
for file in glob.glob(tile_photos_path):
    tile_paths.append(file)

In [4]:
# Import and resize all tiles
from PIL import Image, ImageOps
tiles = []
for path in tile_paths:
    tile = Image.open(path)
    tile = ImageOps.exif_transpose(tile)
    tile = tile.resize(tile_size)
    tiles.append(tile)

In [5]:
# #show some example of images
# for i in range(len(tiles)):
#     plt.figure(i)
#     plt.imshow(tiles[i])

In [6]:
# from PIL import Image, ImageOps

# img = Image.open('photos/tiles\\clock1.JPG')
# img = ImageOps.exif_transpose(img)   # this line was used to change the direction
# img

Some information about mean(axis=0)<br>
[np.mean(a，axis=0)](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)

In [7]:
# Calculate dominant color
colors = []
for tile in tiles:
    mean_color = np.array(tile).mean(axis=0).mean(axis=0) # how to understand (axis=0)
    colors.append(mean_color)

In [8]:
# y = np.array(Image.open(filename).convert('RGB').resize((300,300)))

In [9]:
# Pixelate (resize) main photo
main_photo = Image.open(main_photo_path)
main_photo = ImageOps.exif_transpose(main_photo)

width = int(np.round(main_photo.size[0]/ tile_size[0]))
height = int(np.round(main_photo.size[1]/tile_size[1]))

resized_photo = main_photo.resize((width, height))

In [10]:
# Find closest tile photo for every pixel

# create a KDTree
tree = spatial.KDTree(colors)  # what is KDTree, it might be used to search

#Empty integer array to store indices of tiles
closest_tiles = np.zeros((width, height),dtype=np.uint32)

for i in range(width):
    for j in range(height):
        pixel = resized_photo.getpixel((i,j))
        closest = tree.query(pixel)
        closest_tiles[i,j] = closest[1]

In [11]:
# create an output image
output = Image.new('RGB',main_photo.size)

#Draw tiles
for i in range(width):
    for j in range(height):
        #offset of tile
        x,y = i*tile_size[0],j*tile_size[1]
        #index of tile
        index = closest_tiles[i,j]
        #draw tile
        output.paste(tiles[index],(x,y))
        
#save output
output.save('output.jpg')

In [28]:
# resize the output image
output = Image.open('output.jpg')
resize_output = output.resize((300,300),Image.ANTIALIAS)
resize_output.save('resize_output.jpg')

In [ ]:
# resize the output image
output = Image.open('output.jpg')
resize_output = output.resize((300,300),Image.ANTIALIAS)
resize_output.save('resize_output.jpg')